Part 3

In [2]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306

dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)



def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result



tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series. The database contains information on both. We are not interested in the ‘Masters’. There is also a season called “Canada 6” which is part of the Masters series. Create versions of the chefdetails and judges tables that exclude any records coming from seasons that include the word Masters or Canada.

In [3]:
with engine.connect() as conn:
    chef_df = pd.read_sql(
        sqla.text("SELECT * FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"),
        conn
    )

chef_df


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,386,May Phattanant Thongthong,May Phattanant Thongthong,"Chiang Mai, Thailand",Bangkok,Thailand,NaN,World All Stars,20,US,14,POC,Executive Chef/Co-Partner,Female
318,387,Begona Rodrigo,Begona Rodrigo,"Valencia, Spain",Valencia,Spain,NaN,World All Stars,20,US,12,None,Owner,Female
319,388,Gabriel Rodriguez,Gabriel Rodriguez,"Iztapalapa, Mexico City",Madrid,Spain,NaN,World All Stars,20,US,2,POC,Executive Chef,Male
320,389,Amar Santana,Amar S.,Dominican Republic / Queens,Santa Ana,California,NaN,World All Stars,20,US,6,POC,Chef/Partner,Male


In [4]:
with engine.connect() as conn:
    judges_df = pd.read_sql(
        sqla.text("SELECT * FROM judges WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"),
        conn
    )

judges_df

,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None
...,...,...,...,...,...,...,...,...,...,...
628,750,World All Stars,20,US,12,Quickfire,Individual,Sam Bompas,None,None
629,751,World All Stars,20,US,12,Elimination,Individual,Jeremy Chan,None,None
630,752,World All Stars,20,US,13,Quickfire,Individual,Greg Marchand,None,None
631,753,World All Stars,20,US,13,Elimination,Individual,Greg Marchand,None,None


Further trim your chefdetails and judges tables by including only the columns of interest from the table descriptions above.

In [5]:
# Function to fetch data from the database and filter columns
def fetch_and_filter(engine, table_name, columns, condition):
    with engine.connect() as conn:
        query = f"SELECT {', '.join(columns)} FROM {table_name} WHERE {condition};"
        df = pd.read_sql(sqla.text(query), conn)
    return df

# Define columns of interest for chefdetails
chefdetails_columns = ['season', 'seasonNumber', 'name', 'placement', 'gender']

# Fetch and filter chefdetails table
chefdetails_df = fetch_and_filter(
    engine,
    'chefdetails',
    chefdetails_columns,
    "season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%'"
)

# Print the first few rows of the resulting DataFrame
print("Chef Details (Filtered Columns):")
print(chefdetails_df.head())

# Define columns of interest for judges
judges_columns = ['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge', 'competedOnTC']

# Fetch and filter judges table
judges_df = fetch_and_filter(
    engine,
    'judges',
    judges_columns,
    "season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%'"
)

# Print the first few rows of the resulting DataFrame
print("\nJudges (Filtered Columns):")
print(judges_df.head())


Chef Details (Filtered Columns):
                season  seasonNumber            name  placement  gender
0  All Stars: New York             8   Richard Blais          1    Male
1  All Stars: New York             8   Mike Isabella          2    Male
2  All Stars: New York             8  Antonia Lofaso          3  Female
3  All Stars: New York             8   Tiffany Derry          4  Female
4  All Stars: New York             8      Carla Hall          5  Female

Judges (Filtered Columns):
                season  seasonNumber  episode challengeType        guestJudge  \
0  All Stars: New York             8        1     Quickfire     Tom Colicchio   
1  All Stars: New York             8        1   Elimination  Anthony Bourdain   
2  All Stars: New York             8        2     Quickfire         Joe Jonas   
3  All Stars: New York             8        2   Elimination         Katie Lee   
4  All Stars: New York             8        3     Quickfire       David Chang   

  competedOnTC  
0  

In [6]:
# Define the query to extract unique seasons and seasonNumbers
unique_seasons_query = """
SELECT DISTINCT season, seasonNumber
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';
"""

# Fetch the data from the database
unique_seasons_df = df_select(unique_seasons_query, engine)

# Print the resulting DataFrame
print("Unique Seasons and Season Numbers:")
print(unique_seasons_df)


Retrieved 20 records
Unique Seasons and Season Numbers:
                 season  seasonNumber
0   All Stars: New York             8
1        All-Stars L.A.            17
2                Boston            12
3            California            13
4            Charleston            14
5               Chicago             4
6                  D.C.             7
7              Colorado            15
8               Houston            19
9              Kentucky            16
10            Las Vegas             6
11          Los Angeles             2
12                Miami             3
13          New Orleans            11
14             New York             5
15             Portland            18
16        San Francisco             1
17              Seattle            10
18                Texas             9
19      World All Stars            20


4. Answer the following questions using your tables so far.

a)Among all chef contestants, how many are male and how many are female?


In [8]:
# Define the query to count male and female chef contestants
gender_count_query = """
SELECT gender, COUNT(*) AS count
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%'
GROUP BY gender;
"""

# Fetch the data from the database
gender_count_df = df_select(gender_count_query, engine)

# Print the resulting DataFrame
print("Gender Count of Chef Contestants:")
print(gender_count_df)


Retrieved 2 records
Gender Count of Chef Contestants:
   gender  count
0    Male    170
1  Female    152


b) Among all winners (placement==1) how many are male and how many are female?

In [9]:
# Define the query to count male and female winners
gender_count_winners_query = """
SELECT gender, COUNT(*) AS count
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%' AND placement = 1
GROUP BY gender;
"""

# Fetch the data from the database
gender_count_winners_df = df_select(gender_count_winners_query, engine)

# Print the resulting DataFrame
print("Gender Count of Winners:")
print(gender_count_winners_df)


Retrieved 2 records
Gender Count of Winners:
   gender  count
0    Male     14
1  Female      6


c) Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?

In [10]:
# Define the query to count male and female top 3 finalists
gender_count_top3_query = """
SELECT gender, COUNT(*) AS count
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%' AND placement IN (1, 2, 3)
GROUP BY gender;
"""

# Fetch the data from the database
gender_count_top3_df = df_select(gender_count_top3_query, engine)

# Print the resulting DataFrame
print("Gender Count of Top 3 Finalists:")
print(gender_count_top3_df)


Retrieved 2 records
Gender Count of Top 3 Finalists:
   gender  count
0    Male     36
1  Female     25


d) Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?

In [11]:
# Display the counts of females in the top 3 and females who won
print("Count of Females in Top 3:")
print(gender_count_top3_df)

print("\nCount of Females Who Won:")
print(gender_count_winners_df)


Count of Females in Top 3:
   gender  count
0    Male     36
1  Female     25

Count of Females Who Won:
   gender  count
0    Male     14
1  Female      6


If the count of females in the top 3 is significantly higher than the count of females who won, it might indicate that women who place in the top 3 are less likely than expected to ultimately win the competition.
If the counts are similar, it suggests that women who reach the top 3 have a similar likelihood of winning as compared to men.
If the count of females who won is higher than expected based on the count of females in the top 3, it suggests that women who reach the top 3 are more likely than expected to win.
To draw more definitive conclusions or conduct statistical analysis, you might want to perform a hypothesis test or calculate proportions based on the overall population of contestants. Without additional statistical analysis, it's challenging to definitively determine whether the observed difference is statistically significant or if it could be due to random variation in the data.

4. Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.

In [13]:
# Define the query to retrieve information about first-place winners and whether they served as a guest judge
first_place_winners_query = """
SELECT
    c.name AS winner_name,
    c.season AS winning_season,
    CASE WHEN j.guestJudge IS NOT NULL THEN 'Yes' ELSE 'No' END AS served_as_guest_judge
FROM
    chefdetails c
LEFT JOIN
    judges j ON c.season = j.season AND c.seasonNumber = j.seasonNumber AND j.episode = 1
WHERE
    c.placement = 1 AND c.season NOT LIKE '%Masters%' AND c.season NOT LIKE '%Canada%';
"""

# Fetch the data from the database
first_place_winners_df = df_select(first_place_winners_query, engine)

# Print the resulting DataFrame
print("First Place Winners with Guest Judge Information:")
print(first_place_winners_df)


Retrieved 32 records
First Place Winners with Guest Judge Information:
             winner_name       winning_season served_as_guest_judge
0          Richard Blais  All Stars: New York                   Yes
1          Richard Blais  All Stars: New York                   Yes
2           Melissa King       All-Stars L.A.                   Yes
3                Mei Lin               Boston                   Yes
4                Mei Lin               Boston                   Yes
5            Jeremy Ford           California                    No
6      Brooke Williamson           Charleston                   Yes
7      Brooke Williamson           Charleston                   Yes
8        Stephanie Izard              Chicago                   Yes
9        Stephanie Izard              Chicago                   Yes
10          Kevin Sbraga                 D.C.                   Yes
11          Joseph Flamm             Colorado                   Yes
12          Joseph Flamm             Colorado

5. Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [14]:
# Define the query to retrieve contestants who were not first-place winners but served as a guest judge
guest_judge_contestants_query = """
SELECT DISTINCT
    c.name AS contestant_name
FROM
    chefdetails c
JOIN
    judges j ON c.name = j.guestJudge
WHERE
    c.placement <> 1 AND c.season NOT LIKE '%Masters%' AND c.season NOT LIKE '%Canada%';
"""

# Fetch the data from the database
guest_judge_contestants_df = df_select(guest_judge_contestants_query, engine)

# Print the resulting DataFrame
print("Contestants who were not First Place Winners but served as Guest Judges:")
print(guest_judge_contestants_df)


Retrieved 32 records
Contestants who were not First Place Winners but served as Guest Judges:
       contestant_name
0    Brooke Williamson
1        Richard Blais
2       Antonia Lofaso
3        Carlos Gaytan
4         Nicole Gomes
5           Sam Talbot
6      Bryan Voltaggio
7        Mike Isabella
8     Spike Mendelsohn
9         Dawn Burrell
10        Melissa King
11         Nini Nguyen
12       Tiffany Derry
13      Kwame Onwuachi
14    Nyesha Arrington
15      Shota Nakajima
16   Adrienne Cheatham
17      Sheldon Simeon
18     Karen Akunowicz
19        Nina Compton
20          Edward Lee
21        Betty Fraser
22     Michael Midgley
23        Amar Santana
24        Carrie Baird
25          Dale Talde
26     Gregory Gourdet
27       Lee Anne Wong
28      Miguel Morales
29    Stephen Asprinio
30  Josie Smith-Malave
31      Stefan Richter
